<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 1: Standardized Test Analysis

---

## Data Import and Cleaning

### Data Sources

All origial data sets are included in the [`data`](./data/) directory for this project.

* [`act_2017.csv`](./data/act_2017.csv): 2017 ACT Scores by State
* [`act_2018.csv`](./data/act_2018.csv): 2018 ACT Scores by State
* [`act_2019.csv`](./data/act_2019.csv): 2019 ACT Scores by State
* [`sat_2017.csv`](./data/sat_2017.csv): 2017 SAT Scores by State
* [`sat_2018.csv`](./data/sat_2018.csv): 2018 SAT Scores by State
* [`sat_2019.csv`](./data/sat_2019.csv): 2019 SAT Scores by State
* [`sat_2019_by_intended_college_major.csv`](./data/sat_2019_by_intended_college_major.csv): 2019 SAT Scores by Intended College Major
* [`sat_act_by_college.csv`](./data/sat_act_by_college.csv): Ranges of Accepted ACT & SAT Student Scores by Colleges

### Coding Challenges

1. Manually calculate mean:

In [1]:
# Code: Calculate the Mean

def find_mean(data):
    return sum(data)/len(data)

find_mean([1, 2, 3, 4, 5])

3.0

2. Manually calculate standard deviation:

    The formula for standard deviation is below:

    $$\sigma = \sqrt{\frac{1}{n}\sum_{i=1}^n(x_i - \mu)^2}$$

In [2]:
# Code: Calculate the Standard Deviation
def find_std(data):
    sq_differences = [(x - find_mean(data))**2 for x in data]
    variance = (sum(sq_differences)) / (len(data) - 1)
    return variance**.5

In [3]:
find_std([-5, 1, 8, 7, 2])

5.224940191045253

3. Data cleaning function:
    
    Write a function that takes in a string that is a number and a percent symbol (ex. '50%', '30.5%', etc.) and converts this to a float that is the decimal approximation of the percent.

In [4]:
# Code: Percent string to decimal float
def to_decimal(percent):
    return (float(percent.split('%')[0]))/100

In [5]:
to_decimal('30.5%')

0.305

4. Data cleaning function: lowercase column names

In [6]:
# Return lowercase column names
def lowercaseify(df):
	new_cols = [column.lower().replace(" ", "_") for column in df.columns]
	old_cols = df.columns
	to_map = dict(zip(df.columns, new_cols))
	df.rename(columns = to_map, inplace = True)

	print(df.columns)

--- 
# Part 2

Part 2 requires knowledge of Pandas, EDA, data cleaning, and data visualization.

---

*All libraries used should be added here*

In [7]:
# Imports:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Import and Cleaning

### Data Import & Cleaning

1. Display the data: print the first 5 rows of each dataframe to your Jupyter notebook.
2. Check for missing values.
3. Check for any obvious issues with the observations (keep in mind the minimum & maximum possible values for each test/subtest).
4. Fix any errors you identified in steps 2-3.
5. Display the data types of each feature.
6. Fix any incorrect data types found in step.
7. Rename Columns.
8. Drop unnecessary rows (if needed).
9. Merge dataframes that can be merged. (pd.concat(axis, row))
10. Perform any additional cleaning that you feel is necessary.
11. Save your cleaned and merged dataframes as csv files.

#### Import libraries

In [8]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

#### Load ACT data from 2017, 2018, and 2019.

In [9]:
# Read in data
act_2017 = pd.read_csv('../data/act_2017.csv')
act_2018 = pd.read_csv('../data/act_2018.csv')
act_2019 = pd.read_csv('../data/act_2019.csv')

In [10]:
# Check shape of data frames
print(f'ACT 2017: {act_2017.shape}')
print(f'ACT 2018: {act_2018.shape}')
print(f'ACT 2019: {act_2019.shape}')

ACT 2017: (52, 7)
ACT 2018: (52, 3)
ACT 2019: (52, 3)


#### Merge ACT Data
1. Create year columns for each data frame
2. Merge on shared columns
3. Convert column names to lowercase and reorder 

In [11]:
# Drop unrelated columns
def drop_unrelated(df, related_df):
	for name in df.columns:
		if name not in related_df.columns:
			df.drop(columns=name, inplace=True) 
			print(f"{name} column dropped")

In [12]:
drop_unrelated(act_2017, act_2018)

English column dropped
Math column dropped
Reading column dropped
Science column dropped


In [13]:
# Create columns with single unique value
def single_value_column(df, name, value):
    df[name] = value
    return df[name]

In [14]:
#Create year columns
years = [year for year in range(2017,2019+1)]
act_dfs = [act_2017, act_2018, act_2019]

for i in range(3):
    single_value_column(act_dfs[i], 'year', years[i])
    print(act_dfs[i].head(1))

      State Participation Composite  year
0  National           60%      21.0  2017
     State Participation  Composite  year
0  Alabama          100%       19.1  2018
     State Participation  Composite  year
0  Alabama          100%       18.9  2019


In [15]:
# Merge ACT data on shared columns
df_act = pd.concat(act_dfs, ignore_index=True)
lowercaseify(df_act)

Index(['state', 'participation', 'composite', 'year'], dtype='object')


Confirmed that sum of concatenated rows is 52*3 = 156 and number of shared columns plus the year column is 4.

In [16]:
# Reorder columns
df_act = df_act[['year', 'state', 'participation', 'composite']]

In [17]:
df_act.head()

,year,state,participation,composite
0,2017,National,60%,21.0
1,2017,Alabama,100%,19.2
2,2017,Alaska,65%,19.8
3,2017,Arizona,62%,19.7
4,2017,Arkansas,100%,19.4


#### Clean ACT data
1. Find duplicate and missing values
2. Check and data types

In [18]:
df_act.loc[df_act.state == 'National']

,year,state,participation,composite
0,2017,National,60%,21.0
155,2019,National,52%,20.7


In [19]:
df_act.drop(index=[0,155],inplace=True)

In [20]:
# Check for duplicate values in State column
len(sorted(df_act['state'].unique()))  #53

52

In [21]:
# Replace mispelled value in State column
df_act['state'].replace('District of columbia', 'District of Columbia', inplace=True)
len(sorted(df_act['state'].unique()))  #52

51

In [22]:
# Convert Participation to decimal so datatype is float
df_act['participation'] = df_act['participation'].apply(to_decimal)

In [23]:
# Find and replace string values in Composite
# Change data type to float
#df_act['composite'].unique()
df_act['composite'] = df_act['composite'].replace('20.2x', 20.2)
df_act['composite'] = df_act['composite'].apply(lambda x: float(x))

#### Export Cleaned ACT Data

In [24]:
df_act.to_csv('../data-clean/act_clean.csv', index=False)

#### Load SAT Data for 2017, 2018, 2019

In [25]:
# Read in data
sat_2017 = pd.read_csv('../data/sat_2017.csv')
sat_2018 = pd.read_csv('../data/sat_2018.csv')
sat_2019 = pd.read_csv('../data/sat_2019.csv')

In [26]:
# Check shape of data frames
print(f'SAT 2017: {sat_2017.shape}')
print(f'SAT 2018: {sat_2018.shape}')
print(f'SAT 2019: {sat_2019.shape}')

SAT 2017: (51, 5)
SAT 2018: (51, 5)
SAT 2019: (53, 5)


Address 53 row values for 2019.

In [27]:
states_2019 = sat_2019['State'].unique()
states_2018 = sat_2018['State'].unique()

[i for i in states_2019 if i not in states_2018]  # Puerto Rico, Virgin Islands

['Puerto Rico', 'Virgin Islands']

In [28]:
sat_2019 = sat_2019.loc[(sat_2019['State'] != 'Puerto Rico')]
sat_2019 = sat_2019.loc[(sat_2019['State'] != 'Virgin Islands')]

In [29]:
sat_2019.shape

(51, 5)

#### Merge SAT Data
1. Create year columns for each data frame
2. Merge on shared columns
3. Convert column names to lowercase and reorder

In [30]:
# Create year columns
sat_2017['year'] = 2017
sat_2018['year'] = 2018
sat_2019['year'] = 2019

In [31]:
# Create shared column names
sat_2017.rename(columns={'Evidence-Based Reading and Writing': 'EBRW'}, inplace=True)
sat_2018.rename(columns={'Evidence-Based Reading and Writing': 'EBRW'},  inplace=True)
sat_2019.rename(columns={'Participation Rate': 'Participation'}, inplace=True)

In [32]:
df_sat = pd.concat([sat_2017,sat_2018,sat_2019], ignore_index=True)
df_sat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   State          153 non-null    object
 1   Participation  153 non-null    object
 2   EBRW           153 non-null    int64 
 3   Math           153 non-null    int64 
 4   Total          153 non-null    int64 
 5   year           153 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 7.3+ KB


In [33]:
lowercaseify(df_sat)

Index(['state', 'participation', 'ebrw', 'math', 'total', 'year'], dtype='object')


In [34]:
# Drop  
df_sat = df_sat[['year', 'state', 'participation', 'total']]
df_sat.head(0)

,year,state,participation,total


#### Clean SAT data
1. Find duplicate and missing values
2. Check and change data types

In [35]:
df_sat.isna().sum()  # 0

year             0
state            0
participation    0
total            0
dtype: int64

In [36]:
df_sat['participation'] = df_sat['participation'].apply(to_decimal)

In [37]:
df_sat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           153 non-null    int64  
 1   state          153 non-null    object 
 2   participation  153 non-null    float64
 3   total          153 non-null    int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 4.9+ KB


#### Export Cleaned SAT Data

In [56]:
df_act.participation.describe()

count    154.000000
mean       0.615260
std        0.336501
min        0.060000
25%        0.290000
50%        0.660000
75%        1.000000
max        1.000000
Name: participation, dtype: float64

In [59]:
df_act.loc[df_act.participation<=.06] #24.3

,year,state,participation,composite
123,2019,Maine,0.06,24.3


In [60]:
df_act.loc[df_act.participation>=1] #21.5

,year,state,participation,composite
1,2017,Alabama,1.0,19.2
4,2017,Arkansas,1.0,19.4
6,2017,Colorado,1.0,20.8
18,2017,Kentucky,1.0,20.0
19,2017,Louisiana,1.0,19.5
24,2017,Minnesota,1.0,21.5
25,2017,Mississippi,1.0,18.6
26,2017,Missouri,1.0,20.4
27,2017,Montana,1.0,20.3
29,2017,Nevada,1.0,17.8


In [ ]:
df_sat.to_csv('../data-clean/sat_clean.csv', index=False)

#### Load data for SAT and ACT scores by college 

In [ ]:
# Read in data
colleges = pd.read_csv('../data/sat_act_by_college.csv')
colleges.shape

In [ ]:
colleges.info()

#### Clean data from colleges
1. Find and replace missing and unwanted values
2. Change data types
3. Rename and drop columns

In [ ]:
# Find and replace zero-width values in Schools
#sorted(colleges['School'].unique())
def remove_value(series, value):
    return [text.replace(value, '') for text in series]

colleges['School'] = remove_value(colleges['School'], '\u200b')

In [ ]:
# Replace Test Optional? values with 1 or 0
colleges['Test Optional?'] = colleges['Test Optional?'].apply(lambda x: 1 if x == 'Yes' else 0)

In [ ]:
# Fill empty cells
colleges.fillna(np.nan, inplace=True)

In [ ]:
# More descriptive values for years of test optional policy starting in 2021
a = colleges['Applies to Class Year(s)'].unique()
b = [1, 2, 0, 4, 3, 5, np.nan]
map_years = dict(zip(a, b))

colleges['Applies to Class Year(s)'].replace(map_years, inplace=True)

In [ ]:
#colleges['Policy Details']

`colleges['Policy Details']` contains long strings of text containing info on nuances of their test optional or test required policies.

In [ ]:
# Convert Accept Rate to float column
colleges['Accept Rate'] = colleges['Accept Rate'].apply(to_decimal)

In [ ]:
# Create SAT 25th Percentile and 75th Percentile Columns

#Remove non-int values
colleges['SAT Total 25th-75th Percentile'] = remove_value(colleges['SAT Total 25th-75th Percentile'], '\u200b\u200b ')
colleges['SAT Total 25th-75th Percentile'].replace('--', np.nan, inplace=True)

split_sat_scores = colleges['SAT Total 25th-75th Percentile'].str.split('-',expand=True)
colleges['sat_q1'] = split_sat_scores[0].astype('float64')
colleges['sat_q3'] = split_sat_scores[1].astype('float64')

colleges.drop(columns='SAT Total 25th-75th Percentile', inplace=True)
colleges.head(1)

In [ ]:
# Create ACT 25th Percentile and 75th Percentile Columns

#Remove non-int values
colleges['ACT Total 25th-75th Percentile'] = remove_value(colleges['ACT Total 25th-75th Percentile'], '\u200b\u200b ')
colleges['ACT Total 25th-75th Percentile'].replace('--', np.nan, inplace=True)

split_act_scores = colleges['ACT Total 25th-75th Percentile'].str.split('-',expand=True)
colleges['act_q1'] = split_act_scores[0].astype('float64')
colleges['act_q3'] = split_act_scores[1].astype('float64')

colleges.drop(columns='ACT Total 25th-75th Percentile', inplace=True)
colleges.head(1)

In [ ]:
lowercaseify(colleges)

In [ ]:
colleges.rename({'applies_to_class_year(s)': 'policy_period'}, axis=1, inplace=True)

In [ ]:
colleges.info()

#### Export Cleaned Colleges Data

In [ ]:
colleges.to_csv('../data-clean/colleges_clean.csv', index=False)